In [1]:
import pandas as pd
import numpy as np
import time
from transferEntropy import *

/home/rute/.conda/envs/pesquisa/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [7]:
def generate_df_valid_corrs(df, limit):
    df_valid = pd.DataFrame(np.zeros([df.shape[0], df.shape[1]], dtype=int), columns = df.columns, index= df.columns)
    for row in df.columns:
        for col in df.columns:
            if df[row][col] > limit:
                df_valid[row][col] = 1
                
    return df_valid 

In [19]:
def app_roll_mean(df, window):
    roll  = df.rolling(window).mean() 
    roll.dropna(inplace=True)
    roll = roll.round(decimals=0).copy()
    roll.reset_index(drop=True, inplace=True)
    return roll.copy()

In [29]:
def eval_tes(df_current, path):
    all_cen_normal = ['k%d_l%d_h%d' %(x,y,z) for x,y in zip([1,2,3,4,5,6],[1,2,3,4,5,6])  for z in [1,2,3]]
    for i in np.arange(1,7):
        for j in np.arange(1,4):
            te_matrix = transferEntropy_case(df_current, 0.99, i,i,j)
            t = np.mean(te_matrix) + 3*np.std(te_matrix)
            df_te = pd.DataFrame(te_matrix, columns = df_current.columns, index= df_current.columns)
            generate_df_valid_corrs(df_te, t).to_csv(path+all_cen_normal[(j-1) + 3*(i-1)]+"_sel_corrs.csv")
            df_te.to_csv(path+all_cen_normal[(j-1) + 3*(i-1)]+"_corrs.csv")

In [21]:
cen_normal = "cenarios/cen_nova_versao/normal/"
cen_media2 = "cenarios_/cen_nova_versao/media2/"
cen_media5 = "cenarios/cen_nova_versao/media5/"

In [2]:
#Receving data
dist6_novo = pd.read_csv("dist6_3horas_sig/alm_seq.csv")
dist = dist6_novo[['xmeas%02d_low' % x for x in [1,2,3,8,9,21]] + ['xmeas%02d_high' % x for x in [1,2,3,8,9,21]]]
dist6_novo.drop('tout', axis=1, inplace=True)
dist = dist6_novo[['xmeas%02d_low' % x for x in [1,2,3,8,9,21]] + ['xmeas%02d_high' % x for x in [1,2,3,8,9,21]]]

In [30]:
#cen normal
eval_tes(dist, cen_normal)

530.9326019999999


In [31]:
#Cen mean 2
df_mean2 = app_roll_mean(dist,2)
eval_tes(df_mean2, cen_media2)

569.8660880000007


In [32]:
#cen mean5 
df_mean5 = app_roll_mean(dist,5)
eval_tes(df_mean5, cen_media5)

575.5115139999998
